<a href="https://colab.research.google.com/github/mayankcircle/Tensorflow-2.0/blob/main/Tensorflow_Tutorial_3_(Keras_Sequential_and_Functional_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Libraries

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [7]:
tf.__version__

'2.3.0'

# Load MNIST Dataset

In [3]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [24]:
# classees
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [11]:
X_train.shape # we have 60000 gray-scale images (only one channel) of size 28 x 28 in train data

(60000, 28, 28)

In [4]:
# faltten out the images
X_train = X_train.reshape(-1,28*28)
X_train.shape

(60000, 784)

In [15]:
X_train.dtype # data type is int 8 bit so possible values will be from 0 to 255

dtype('uint8')

In [5]:
# we can change the data type to float32 and normalize the values to have 0 to 1 for fast training
X_train = X_train.astype("float32") / 255.0
X_train.dtype

dtype('float32')

In [6]:
# We also do the same for X_test (test data)
X_test = X_test.reshape(-1,28*28).astype("float32")/255.0
X_test.shape

(10000, 784)

In [ ]:
# OPTIONAL : Convert numpy array to Tensorflow Tensor
# Though Conversion from Numpy array to TEnsor will be done internally automatically by tensorflow, we can also achieve this by-
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)

# Build Basic Neural Network using Sequential API

**Sequential API is very convenient but not very flexible. This is only means for one input to one output mapping. If we want 2nd case then we go for Functional API.**

In [31]:
model = keras.Sequential(
    [
     keras.Input(shape=(28*28)),
     layers.Dense(512, activation = "relu"),
     layers.Dense(256, activation = "relu"),
     layers.Dense(10)
    ]
)

We are not applying Softmax in output layer of Neural Network because Softmax will be done from the Loss Function. So we raw Logits into that.

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


Since we have labels as integers, we can use **SparseCategoricalCrossentropy** loss function. Here, we dont need to convert labels into one-hot vectors. If we convert labels into one-hot vectors then we can use **CategoricalCrossentropy** loss function.

**we are using "accuracy" as a metric to track be keras during training. It is nothing to do with optimizing the loss function, just for tracking.**

In [33]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), # we are passing the logits.
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

# Train Neural Network

In [34]:
model.fit(X_train,y_train,batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1877 - accuracy: 0.9432
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0802 - accuracy: 0.9753
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0542 - accuracy: 0.9822
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0413 - accuracy: 0.9868
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0336 - accuracy: 0.9891


# Evaluate Model

In [32]:
model.evaluate(X_test,y_test,batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 0.0804 - accuracy: 0.9795


[0.08040821552276611, 0.9794999957084656]

# Build Basic Neural Network using Functional API

It is bit more flexible, meant for multiple input to multiple output mapping.

In [47]:
inputs = keras.Input(shape=(28*28))
x = layers.Dense(512, activation="relu",name="First_Layer")(inputs)
x = layers.Dense(256, activation="relu",name="Seccond_Layer")(x)
outputs = layers.Dense(10, activation="softmax",name="Output_Layer")(x)

# build model
model = keras.Model(inputs=inputs, outputs=outputs)



In [48]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
First_Layer (Dense)          (None, 512)               401920    
_________________________________________________________________
Seccond_Layer (Dense)        (None, 256)               131328    
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), # here we are passing the softmax op
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

In [42]:
model.fit(X_train,y_train,batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1859 - accuracy: 0.9433
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0786 - accuracy: 0.9754
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0563 - accuracy: 0.9821
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0404 - accuracy: 0.9872
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0330 - accuracy: 0.9893


In [43]:
model.evaluate(X_test,y_test,batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 0.0737 - accuracy: 0.9791


[0.07366567850112915, 0.9790999889373779]

# How to get an output of specific layer of Neural Network?

The below technique will be working for both Sequential API as well as Functional API.

1st we again rebuild a sequential model

In [ ]:
model = keras.Sequential(
    [
     keras.Input(shape=(28*28)),
     layers.Dense(512, activation = "relu"),
     layers.Dense(256, activation = "relu"),
     layers.Dense(10)
    ]
)

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), # we are passing the logits.
    optimizer = keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

In [ ]:
model.fit(X_train,y_train,batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1877 - accuracy: 0.9432
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0802 - accuracy: 0.9753
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0542 - accuracy: 0.9822
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0413 - accuracy: 0.9868
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0336 - accuracy: 0.9891


In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [36]:
# we can get output of each layers (Recommended way)
model = keras.Model(inputs=model.inputs,
                    outputs=[layer.output for layer in model.layers])

In [37]:
features = model.predict(X_train)

for feature in features:
    print(feature.shape)

(60000, 512)
(60000, 256)
(60000, 10)


In [40]:
# getting the logits value (last layer output)
print("Last layer shape : ",features[-1].shape)
print("Get the last layer output (logits) of 1st sample only")
features[-1][0]

Last layer shape :  (60000, 10)
Get the last layer output (logits) of 1st sample only


array([-28.69341   , -10.879023  , -15.435382  ,  11.1008215 ,
       -25.210272  ,  22.408262  , -14.360717  , -15.948825  ,
        -8.335175  ,  -0.84735394], dtype=float32)

**So logits are basically raw output of Neural Network without any activation function such as softmax etc. THey are also not the probabilities also, they are just the values (any real number)**

In [ ]:
# getting the second last layer output (layers[-2])

model = keras.Model(inputs = model.inputs,
                    outputs=[model.layers[-2].output])

# OR we can aslo get the partiicular layer by its name
"""
model = keras.Model(inputs = model.inputs,
                    outputs=[model.get_layer("dense_1").output])
"""

In [29]:
features = model.predict(X_train)
features.shape

(60000, 256)

In [30]:
# if we get 2nd layer output, the model will be changed to have from input to 2nd llast layer only.
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
Total params: 533,248
Trainable params: 533,248
Non-trainable params: 0
_________________________________________________________________
